# 패키지 선언

In [1]:
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import requests
import re
import pandas as pd
import numpy as np
import json
from random import *
from tqdm.notebook import tqdm

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import UnexpectedAlertPresentException

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--no-referrers')
options.add_argument('user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Safari/537.36')

executable_path = '/opt/google/chrome/chromedriver' # chrome driver의 경로
driver = webdriver.Chrome(executable_path=executable_path, options=options)

/home/kpkim/.local/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# isbn 데이터 불러오기 - 알라딘 베스트셀러 2020

In [2]:
best_2020 = pd.read_csv('./book_data/nationallib/bestseller_aladin/NL_BO_BEST_BOOK_HISTORY_ARCHIVE_202012.csv', low_memory=False)
best_isbns = best_2020.loc[:, 'isbn13'].tolist()

len(best_isbns)

190852

# 네이버 책 크롤링

In [3]:
isbn = best_isbns[48]
isbn

'9788959138524'

In [4]:
def naver_crawl(isbn):
    main_page = 'https://book.naver.com/'
    driver.get(main_page)
    driver.implicitly_wait(3)
    result = {'ISBN13' : isbn}

    search_box = driver.find_element_by_class_name('snb_search_box')
    search_btn = search_box.find_element_by_id('search_btn')
    input_box = search_box.find_element_by_name('query')
    input_box.send_keys(isbn)
    search_btn.click()    # isbn 검색 후 검색버튼 클릭
    
    try:
        result_list = driver.find_element_by_id('searchBiblioList')
        top_item = result_list.find_element_by_tag_name('li')
        thumbnail = top_item.find_element_by_tag_name('div')
        thumbnail.click()    # 검색 리스트 첫번째 아이템 - 썸내일 클릭, 새 탭에서 책 정보 페이지 열기

        driver.switch_to.window(driver.window_handles[1])    # 책 정보 탭으로 이동
        
        try:
            # book info
            book_info = driver.find_element_by_class_name('book_info')
            pubreview_open = driver.find_element_by_id('pubReviewOpen')   # 출판사 서평 펼치기 버튼 클릭
            pubreview_open.click()

            info_soup = BeautifulSoup(book_info.get_attribute('outerHTML'), 'html.parser')

            title = info_soup.h2.get_text().replace('\xa0', ' - ')   # 책 제목
            result['title'] = title

            book_info_inner = info_soup.find('div', 'book_info_inner')
            rating_div = book_info_inner.find('div', 'txt_desc')
            ratings = rating_div.find_all('strong')
            star_rating = ratings[0].text.replace('점', '').strip()    # 별점
            result['star_rating'] = star_rating
            review_num = ratings[1].text.strip()    # 리뷰 수
            result['review_num'] = review_num
            info_divs = rating_div.find_next_siblings('div', limit=3)
            info_list_a = info_divs[0].text.split('|') + info_divs[1].text.split('|') + info_divs[2].text.split('|')
            info_list = list(map(lambda x: x.strip(), info_list_a))    # 저자, 출판사, 출판일, 페이지, ISBN, 소장 정보
            result['info_list'] = info_list

            try:
                # book content
                whole_content = driver.find_element_by_id('content')
                content_soup = BeautifulSoup(whole_content.get_attribute('outerHTML'), 'html.parser')
                book_intro = content_soup.find(id="bookIntroContent").text    # 책 소개
                result['book_intro'] = book_intro
                author_intro = content_soup.find(id="authorIntroContent").text    # 저자 소개
                result['author_intro'] = author_intro
                content_table = content_soup.find(id="contentContent").text.strip()    # 목차
                result['content_table'] = content_table
                pub_review = content_soup.find(id="pubReviewContent").text    # 출판사 서평
                result['pub_review'] = pub_review
                sections = content_soup.find(class_="history").find_all('li')
                sections_list = [s.text.replace('\n', '').replace('\t', '').replace('>', ' >') for s in sections]    # 이 책이 속한 분야
                result['sections_list'] = sections_list
                together_books = content_soup.find(id="togetherBuyBook").find_all('li')    # 함께 읽은 책 - 알라딘
                together_infos = []
                for item in together_books:
                    together_title1 = item.find_all('dt')[1].text.strip()
                    together_title = re.sub('\(.*\)', '', together_title1).strip()
                    togetherg_author = item.find('dd').text.strip()
                    together_infos.append({'title' : together_title, 'author' : togetherg_author})
                result['together_infos'] = together_infos

            except:
                return result

        except:
            return result
        driver.close()

    except:
        return result
    
    # 검색창으로 돌아오기
    driver.switch_to.window(driver.window_handles[0])
    driver.back()
    driver.quit()
    
    return result

In [6]:
naver_arr = []
for isbn in tqdm(best_isbns[:50]):
    result_ob = naver_crawl(isbn)
    naver_arr.append(result_ob)

naver_df = pd.DataFrame(naver_arr)
naver_df

  0%|          | 0/50 [00:00<?, ?it/s]

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=59198): Max retries exceeded with url: /session/ee37a1c29e6230435b08c2a6f240bbdd/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa18a661710>: Failed to establish a new connection: [Errno 111] Connection refused'))